In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV
from PreproccessingCv import X, y

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
base_models = [
    ('random_forest', RandomForestRegressor(random_state=0)),
    ('gradient_boosting', GradientBoostingRegressor(random_state=0))
]

meta_model = LinearRegression()

rf_param_grid = {
    'n_estimators': [50, 100, 200],    
    'max_depth': [None, 10, 20, 30],    
    'min_samples_split': [2, 5, 10],    
    'min_samples_leaf': [1, 2, 4],      
    'max_features': ['auto', 'sqrt', 'log2']  
}

gb_param_grid = {
    'n_estimators': [50, 100, 200],    
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],             
    'min_samples_split': [2, 5, 10],    
    'min_samples_leaf': [1, 2, 4],      
}

meta_param_grid = {
    'fit_intercept': [True, False],  
    'positive': [True, False],        
}

In [7]:
train_meta_features = np.zeros((X_train.shape[0], len(base_models)))

In [8]:
best_base_models = {}
for i, (name, model) in enumerate(base_models):
    grid_search = GridSearchCV(estimator=model, param_grid=rf_param_grid if name == 'random_forest' else gb_param_grid,
                               cv=3, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train["cv"])
    best_base_models[name] = grid_search.best_estimator_
    print("Best parameters for {}:".format(name), grid_search.best_params_)

    train_meta_features[:, i] = grid_search.best_estimator_.predict(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Best parameters for random_forest: {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best parameters for gradient_boosting: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}


In [9]:
meta_grid_search = GridSearchCV(estimator=meta_model, param_grid=meta_param_grid,
                                cv=3, scoring='r2', n_jobs=-1)
meta_grid_search.fit(train_meta_features, y_train["cv"])
best_meta_model = meta_grid_search.best_estimator_
print("Best parameters for Meta-model (Linear Regression):", meta_grid_search.best_params_)

Best parameters for Meta-model (Linear Regression): {'fit_intercept': False, 'positive': False}


In [10]:
X_test = X_test.values

In [11]:
test_meta_features = np.column_stack(tuple(model.predict(X_test) for model in best_base_models.values()))
meta_predictions = best_meta_model.predict(test_meta_features)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [12]:
r2 = r2_score(y_test, meta_predictions)
print("R^2 Score after Hyperparameter Tuning:", r2)
print("mse:",mean_squared_error(y_test, meta_predictions))

R^2 Score after Hyperparameter Tuning: 0.987684146021708
mse: 0.014597507548455813
